In [1]:
import json
import random
import numpy as np
from Modules.model import NeuralNet
from Modules.nltk_utils import bag_of_words,stem,tokenize


import torch

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
with open("intents.json", "r") as f:
    intents = json.load(f)  

data = torch.load("data.pth")    
data["input_size"]

58

In [3]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
num_classes = data["num_classes"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

In [4]:
model = NeuralNet(input_size, hidden_size, num_classes)
model

NeuralNet(
  (l1): Linear(in_features=58, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [5]:
model.load_state_dict(model_state)

<All keys matched successfully>

In [6]:
model.eval()

NeuralNet(
  (l1): Linear(in_features=58, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [7]:
print("let's chat, type quit to exit")
bot_name = "sam"
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break
    
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)
    
    preds = model(X)
    pred_probs = torch.softmax(preds, dim=1)
    pred_probs_labels = torch.argmax(pred_probs, dim=1)
    
    tag = tags[pred_probs_labels]
    
    #verbose
#     print(f'tensor_probs: {pred_probs}')
#     print(f'labels: {tags}')
#     print(f'probs: {pred_probs.max().item()}')
    
    if pred_probs.max().item() > 0.75:
        for intent in intents["intents"]:
            if intent["tag"] == tag:
                print(f'{bot_name}:{random.choice(intent["responses"])}')
    else:
        print(f"sorry could not understand")
print(f"\n{bot_name} left")

let's chat, type quit to exit
You: hi
sam:Hi there, what can I do for you?
You: quit

sam left
